In [8]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
from pypdf import PdfReader
import pytesseract
from PIL import Image

def extract_text_from_pdf(url):
    visited_urls = set()
    pdf_files = []
    
    def download_pdf(url):
        while True:
            try:
                response = requests.get(url, stream=True)
                response.raise_for_status()
                filename = url.split("/")[-1]
                with open(filename, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                return filename
            except requests.exceptions.ChunkedEncodingError as e:
                print("Error occurred during download, retrying...")
                continue
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
    
    def extract_text_from_image(image_path):
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text
    
    def extract_text_from_pdf_file(pdf_file):
        reader = PdfReader(pdf_file)
        text = ""
        for page_number, page in enumerate(reader.pages):
            text += page.extract_text()
            if not text:
                image_path = pdf_file.replace(".pdf", f"_{page_number}.png")
                os.system(f"convert -density 300 {pdf_file}[{page_number}] {image_path}")
                if os.path.exists(image_path):
                    image_text = extract_text_from_image(image_path)
                    os.remove(image_path)
                    text += image_text
        return text
    
    def process_url(url):
        if url in visited_urls:
            return
        visited_urls.add(url)
        if url and url.startswith("mailto:"):
            return
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        for link in soup.find_all("a"):
            href = link.get("href")
            if href is not None and href.endswith(".pdf"):
                pdf_url = urljoin(url, href)
                pdf_file = download_pdf(pdf_url)
                pdf_files.append(pdf_file)
            else:
                if href is not None and href.startswith(".pdf"):
                    if not href.startswith("tel:"):
                        next_url = urljoin(url, href)
                        process_url(next_url)
    
    process_url(url)
    
    zip_filename = "pdf_text.zip"
    with zipfile.ZipFile(zip_filename, "w") as zip_file:
        for pdf_file in pdf_files:
            text = extract_text_from_pdf_file(pdf_file)
            text_filename = pdf_file.replace(".pdf", ".txt")
            with open(text_filename, "w") as f:
                f.write(text)
            zip_file.write(text_filename)
            os.remove(pdf_file)
            os.remove(text_filename)
    
    return zip_filename    


url = "https://pstu.ru/sveden/education/"
zip_filename = extract_text_from_pdf(url)

KeyboardInterrupt: 